# How to add links to Document

Graph vector stores allow documents to be connected in different ways, allowing different types of relationships between documents to be used in retrieval.

Several common types of relationships can be defined automatically, but in some cases you may want to create custom links. In this example we'll see how to create links directly.

In this guide we will cover:

1. How document links work;
2. How to create links;
3. How to extend a document with links;

## How document links work

Documents in a graph vector store are connected via "links". Links form a bipartite graph between documents and tags: documents are connected to tags, and tags are connected to other documents. When documents are retrieved from a graph vector store, a pair of documents are connected with a depth of one if both documents are connected to the same tag.

Links have a "kind", used to namespace different tag identifiers. For example a link to a keyword might use kind `kw`, while a link to a URL might use kind `url`. This allows the same tag value to be used in different contexts without causing name collisions.

Links are directed - the directionality of links controls how the graph is traversed at retrieval time. For example, given documents A and B, connected by links to tag T:

| A to T | B to T | Result |
|--------|--------|--------|
| outgoing | incoming | Retrieval traverses from A to B |
| incoming | incoming | No traversal from A to B |
| outgoing | outgoing | No traversal from A to B |
| incoming | outgoing | No traversal from A to B |
| bidir | incoming | Retrieval traverses from A to B |
| bidir | outgoing | No traversal from A to B |
| outgoing | bidir | Retrieval traverses from A to B |
| incoming | bidir | No traversal from A to B |

Directed links make it possible to describe relationships such as term references / definitions: term definitions are generally relevant to any documents that use the term, but the full set of documents using a term generally aren't relevant to the term's definition.

## Preliminary

In [ ]:
%pip install -q langchain_community

## How to create links

You can create links using the [Link](https://api.python.langchain.com/en/latest/graph_vectorstores/langchain_core.graph_vectorstores.links.Link.html) class's constructors [.bidir](https://api.python.langchain.com/en/latest/graph_vectorstores/langchain_core.graph_vectorstores.links.Link.html#langchain_core.graph_vectorstores.links.Link.bidir), [.incoming](https://api.python.langchain.com/en/latest/graph_vectorstores/langchain_core.graph_vectorstores.links.Link.html#langchain_core.graph_vectorstores.links.Link.incoming), and [.outgoing](https://api.python.langchain.com/en/latest/graph_vectorstores/langchain_core.graph_vectorstores.links.Link.html#langchain_core.graph_vectorstores.links.Link.outgoing).

In [4]:
from langchain_core.graph_vectorstores.links import Link

link = Link.bidir(kind="location", tag="Paris")
link

Link(kind='location', direction='bidir', tag='Paris')

## Extending documents with links

Now that we know how to create links, let's associate them with some documents. These edges will strengthen the connection between documents that share a keyword when using a graph vector store to retrieve documents.

First, we'll load some text and chunk it into smaller pieces.
Then we'll add a link to each document to link them all together.

In [7]:
from langchain_community.document_loaders import TextLoader
from langchain_core.graph_vectorstores.links import add_links
from langchain_text_splitters import CharacterTextSplitter

loader = TextLoader("state_of_the_union.txt")

raw_documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
documents = text_splitter.split_documents(raw_documents)

for doc in documents:
    add_links(doc, Link.bidir(kind="genre", tag="oratory"))

documents[0]

Document(metadata={'source': 'state_of_the_union.txt', 'links': [Link(kind='genre', direction='bidir', tag='oratory')]}, page_content='Madam Speaker, Madam Vice President, our First Lady and Second Gentleman. Members of Congress and the Cabinet. Justices of the Supreme Court. My fellow Americans.  \n\nLast year COVID-19 kept us apart. This year we are finally together again. \n\nTonight, we meet as Democrats Republicans and Independents. But most importantly as Americans. \n\nWith a duty to one another to the American people to the Constitution. \n\nAnd with an unwavering resolve that freedom will always triumph over tyranny. \n\nSix days ago, Russia’s Vladimir Putin sought to shake the foundations of the free world thinking he could make it bend to his menacing ways. But he badly miscalculated. \n\nHe thought he could roll into Ukraine and the world would roll over. Instead he met a wall of strength he never imagined. \n\nHe met the Ukrainian people. \n\nFrom President Zelenskyy to ev

As you can see, each document's metadata now includes a bidirectional link to the genre "oratory".

The documents can then be added to a graph vector store:

In [ ]:
from langchain_community.graph_vectorstores import CassandraGraphVectorStore

graph_vectorstore = CassandraGraphVectorStore.from_documents(
    documents=documents, embeddings=...
)